# Modelos Metapopulacionais

O conceito de metapopulações pode ser visto como uma maneira relativamente simples de tratar a heterogeneidade em populações sem abandonar o paradigma de populações homogêneas modeladas com Equações diferenciais ordinárias.

![metapops](https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/Metapopulation_%281%29.svg/2560px-Metapopulation_%281%29.svg.png)

#### Exercício 1:

Desenvolva um modelo metapopulacional com dois patches (duas sub-populações) A e B nos quais se propaga uma doença de transmissão direta modelada como SIR. Considere que existe um fluxo migratório entre as cidades.

1. Escreva as equações das duas populações representando as migrações $A \rightarrow B$ como  $k_1$, e $B \rightarrow A$ como $k_2$.
1. Utilizando uma parametrização que não permita equilíbrio endêmico, encontre os equilíbrios $S_A^* + S_B^*$ e $I_A^* + I_B^*$ .
1. Calcule a Matriz Jacobiana do sistema e determine  a estabilidade dos equilíbrios.
1. Crie uma função interativa que nos permita alterar os valores de $k_1$ e $k_2$. Realize simulações com $k_1 = k_2$, $k_1 < k_2$ e $k_1 > k_2$, com $\beta_A \neq \beta_B$

In [1]:
%display typeset

#### Resposta ao item 1:

$$ \dot{S}_A = -\beta_A S_A I_A + k_2 S_B - k_1 S_A$$
$$\dot{I}_A = \beta_A S_A I_A + k_2 I_B - k_1 I_A -\gamma I_A$$
$$\dot{S}_B = -\beta_B S_B I_B + k_1 S_A - k_2 S_B $$
$$\dot{I}_B = \beta_B S_B I_B + k_1 I_A - k_2 I_B -\gamma I_B$$

2) Vamos agora encontrar os equilíbrios do sistema usando o solver do sage:

In [20]:
var('S_A I_A S_B I_B beta_A beta_B R_A R_B gamma k_1 k_2')
#beta_A = 1.2
#beta_B = 1.2
#gamma = 0.5
#k_1 = 0.5
#k_2 = 0.5
solve([-beta_A*S_A*I_A + k_2*S_B - k_1*S_A, 
       beta_A*S_A*I_A+ k_2*I_B - k_1*I_A - gamma*I_A,
       -beta_B*S_B*I_B - k_2*S_B + k_1*S_A,
       beta_B*S_B*I_B- k_2*I_B + k_1*I_A - gamma*I_B,
       gamma*I_A,
       gamma*I_B],[S_A, I_A, S_B, I_B, R_A, R_B])

[[S_A == r4, I_A == 0, S_B == k_1*r4/k_2, I_B == 0, R_A == r5, R_B == r6]]

3) Calculamos a Matriz jacobiana para podermos realizar a análise de estabilidade do equilíbrio encontrado acima

In [21]:
J = jacobian([-beta_A*S_A*I_A + k_2*S_B - k_1*S_A, 
       beta_A*S_A*I_A+ k_2*I_B - k_1*I_A - gamma*I_A,
       -beta_B*S_B*I_B - k_2*S_B + k_1*S_A,
       beta_B*S_B*I_B- k_2*I_B + k_1*I_A - gamma*I_B],[S_A, I_A, S_B, I_B])
J

[       -I_A*beta_A - k_1              -S_A*beta_A                      k_2                        0]
[              I_A*beta_A S_A*beta_A - gamma - k_1                        0                      k_2]
[                     k_1                        0        -I_B*beta_B - k_2              -S_B*beta_B]
[                       0                      k_1               I_B*beta_B S_B*beta_B - gamma - k_2]

Agora Substituímos o valor do equilíbrio na matriz jacobiana:

In [22]:
J_eq = J(S_A=1,I_A=0,S_B=k_1/k_2,I_B=0)
J_eq

[                         -k_1                       -beta_A                           k_2                             0]
[                            0          beta_A - gamma - k_1                             0                           k_2]
[                          k_1                             0                          -k_2               -beta_B*k_1/k_2]
[                            0                           k_1                             0 -gamma + beta_B*k_1/k_2 - k_2]

e também os valores dos parâmetros

In [25]:
J_eq(gamma=1.0,k_1=.5,k_2=.5,beta_A=1.6,beta_B=1.6).eigenvalues()

[3/5, -2/5, -1, 0]

Como o maior autovalor é real e positivo o Equilíbrio é instável

4) Agora vamos criar uma simulação interativa para estudar o efeito da existência de patches na dinâmica do sistema.

In [8]:
def model_2_pop(t, y, params):
    beta_A, beta_B, gamma, k_1, k_2, mu = params
    S_A, I_A, S_B, I_B, R_A, R_B = y
    return [-beta_A*S_A*I_A + k_2*S_B - k_1*S_A, 
       beta_A*S_A*I_A+ k_2*I_B - k_1*I_A - gamma*I_A,
       -beta_B*S_B*I_B - k_2*S_B + k_1*S_A,
       beta_B*S_B*I_B - k_2*I_B + k_1*I_A - gamma*I_B,
       gamma*I_A,
       gamma*I_B]

Agora vamos criar uma versão do modelo anterior, com nascimentos e mortes, ou seja demografia:

$$ \dot{S}_A = -\beta_A S_A I_A + k_2 S_B - k_1 S_A -\mu S_A +\mu(S_A+I_A+R_A)$$
$$\dot{I}_A = \beta_A S_A I_A + k_2 I_B - k_1 I_A -\gamma I_A -\mu I_A$$
$$\dot{R}_A = \gamma I_A-\mu R_A$$
$$\dot{S}_B = -\beta_B S_B I_B + k_1 S_A - k_2 S_B -\mu S_B +\mu(S_B+I_B+R_B)$$
$$\dot{I}_B = \beta_B S_B I_B + k_1 I_A - k_2 I_B -\gamma I_B -\mu I_B$$
$$\dot{R}_B= \gamma I_B-\mu R_B$$

In [26]:
def model_2_pop_demog(t, y, params):
    beta_A, beta_B, gamma, k_1, k_2, mu = params
    S_A, I_A, S_B, I_B, R_A, R_B = y
    return [-beta_A*S_A*I_A + k_2*S_B - k_1*S_A -mu*S_A +mu*(S_A+I_A+R_A), 
       beta_A*S_A*I_A+ k_2*I_B - k_1*I_A - gamma*I_A - mu*I_A,
       -beta_B*S_B*I_B - k_2*S_B + k_1*S_A - mu*S_B+mu*(S_B+I_B+R_B),
       beta_B*S_B*I_B - k_2*I_B + k_1*I_A - gamma*I_B - mu*I_B,
       gamma*I_A-mu*R_A,
       gamma*I_B-mu*R_B]

Agora passamos à instanciação do nosso solver e à implementação de uma função de plotagem.

In [10]:
T=ode_solver()
T.function = model_2_pop
T.algorithm = "rk8pd"#"rk4imp"
from itertools import cycle
labels = [r"$S_A$", "$I_A$", "$S_B$", "$I_B$","$R_A$", "$R_B$"]

def plot_sol(sol):
    plots=None
    c = cycle(['red','blue','green', 'black', 'yellow', 'orange', 'magenta', 'gray', 'pink', 'brown'])
    for i,co in zip(range(len(sol[0][1])),c):
#         co = c.next()
        if plots is None:
            plots = list_plot([(j[0],j[1][i]) for j in sol],color=co, plotjoined=True,      legend_label='%s'%labels[i], alpha=.8, gridlines=true)
        else:
            plots += list_plot([(j[0],j[1][i]) for j in sol],color=co, plotjoined=True,           legend_label='%s'%labels[i], alpha=.8, gridlines=true)
    show(plots)

In [31]:
@interact()
def simula(beta_A=(4.6,(.2,5,.1)), beta_B=(3.6,(.2,5,.1)), gamma=(1.2, 2,.1), k_1=(0.5,(0,1,.1)), k_2=(0.5,(0,1,.1)), mu=(.04,(0, 1,.1)), model=selector(['no demog', 'demog'], buttons=True)):
    if model == "demog":
        T.function = model_2_pop_demog
    else:
        T.function = model_2_pop
    inits = (.9,0.1, 1, 0, 0, 0)
    t_span = [0,30]
    T.ode_solve(t_span,inits,num_points=300, params=(beta_A, beta_B, gamma, k_1, k_2, mu))
    show(inits[0]*k_1/k_2)
    plot_sol(T.solution)

Interactive function <function simula at 0x7f10cda98280> with 7 widgets
  beta_A: FloatSlider(value=4.6, descr…

#### Exercício 2:

Modelos evolutivos em 2 patches.  Adapte o modelo de seleção natural sem mutação, para duas subpopulações, mantendo a generalidade para multiplos tipos.

1. Escreva as equações em forma matricial
2. Encontre os equilíbrios do sistema
3. Escreva uma função interativa para estudar os parâmetros

In [32]:
pretty_print(html('a) Vamos escrever o modelo evolutivo, começando com 4 tipos e 5 localidades, denotadas pelos vetores $X$ e $L$, respectivamente:'))
var('x_1 x_2 x_3 x_4 A B C D E')
X = Matrix(SR,[x_1, x_2, x_3, x_4])
L = Matrix(SR,[A,B,C,D,E])
show("X=", X,", L=",L)

D = Matrix(SR, X.ncols(),L.ncols(), lambda i,j: var('{}{}'.format(X[0,i],L[0,j])))
pretty_print(html("Seja $D$ a matriz com as densidades em cada uma das localidades:"))
show("D=",D)
pretty_print(html("Seja $M$ a matriz com as taxas de migração $k_{ij}$ entre cada uma das localidades:"))
M = Matrix(SR, L.ncols(),L.ncols(), lambda i,j: var('k_{}{}'.format(L[0,i],L[0,j])))
MD = Matrix(SR, L.ncols(),L.ncols(), lambda i,j: 0+ int(i==j)*sum(M[i]))
MLT = Matrix(SR, L.ncols(),L.ncols(), lambda i,j: (1-int(j>=i))*var('k_{}{}'.format(L[0,i],L[0,j])))
MUT = Matrix(SR, L.ncols(),L.ncols(), lambda i,j: (1-int(j<i))*var('k_{}{}'.format(L[0,i],L[0,j])))
M = M-(identity_matrix(5).elementwise_product(M))+MD  #MLT+MD+MUT
show("M=",M)
pretty_print(html(r"Logo, $F=D \times M$ é uma matriz em que cada elemento $f_{ij}$ representa a imigração do tipo $i$ em cada patch $j$ vindo de todos os outros patches:"))
F = D*M
show("F=", F)
pretty_print(html("Seja $f_i$ o vetor de fitness de cada tipo $i$:"))
f_i = Matrix(SR, [1/5, 1/3, 2/5, 1/2])
show('f=', f_i)
pretty_print(html(r"Nosso modelo fica então $\dot{x_{ij}}=f_i \times x_{ij} + F_{ij} - x_{ij}\sum_j M_{ij} -\phi x_i$:"))

a) Vamos escrever o modelo evolutivo, começando com 4 tipos e 5 localidades, denotadas pelos vetores <script type="math/tex">X</script> e <script type="math/tex">L</script>, respectivamente:

'X=' [x_1 x_2 x_3 x_4] ', L=' [A B C D E]

Seja <script type="math/tex">D</script> a matriz com as densidades em cada uma das localidades:

'D=' [x_1A x_1B x_1C x_1D x_1E]
[x_2A x_2B x_2C x_2D x_2E]
[x_3A x_3B x_3C x_3D x_3E]
[x_4A x_4B x_4C x_4D x_4E]

Seja <script type="math/tex">M</script> a matriz com as taxas de migração <script type="math/tex">k_{ij}</script> entre cada uma das localidades:

...

Logo, <script type="math/tex">F=D \times M</script> é uma matriz em que cada elemento <script type="math/tex">f_{ij}</script> representa a imigração do tipo <script type="math/tex">i</script> em cada patch <script type="math/tex">j</script> vindo de todos os outros patches:

'F=' [(k_AA + k_AB + k_AC + k_AD + k_AE)*x_1A + k_BA*x_1B + k_CA*x_1C + k_DA*x_1D + k_EA*x_1E k_AB*x_1A + (k_BA + k_BB + k_BC + k_BD + k_BE)*x_1B + k_CB*x_1C + k_DB*x_1D + k_EB*x_1E k_AC*x_1A + k_BC*x_1B + (k_CA + k_CB + k_CC + k_CD + k_CE)*x_1C + k_DC*x_1D + k_EC*x_1E k_AD*x_1A + k_BD*x_1B + k_CD*x_1C + (k_DA + k_DB + k_DC + k_DD + k_DE)*x_1D + k_ED*x_1E k_AE*x_1A + k_BE*x_1B + k_CE*x_1C + k_DE*x_1D + (k_EA + k_EB + k_EC + k_ED + k_EE)*x_1E]
[(k_AA + k_AB + k_AC + k_AD + k_AE)*x_2A + k_BA*x_2B + k_CA*x_2C + k_DA*x_2D + k_EA*x_2E k_AB*x_2A + (k_BA + k_BB + k_BC + k_BD + k_BE)*x_2B + k_CB*x_2C + k_DB*x_2D + k_EB*x_2E k_AC*x_2A + k_BC*x_2B + (k_CA + k_CB + k_CC + k_CD + k_CE)*x_2C + k_DC*x_2D + k_EC*x_2E k_AD*x_2A + k_BD*x_2B + k_CD*x_2C + (k_DA + k_DB + k_DC + k_DD + k_DE)*x_2D + k_ED*x_2E k_AE*x_2A + k_BE*x_2B + k_CE*x_2C + k_DE*x_2D + (k_EA + k_EB + k_EC + k_ED + k_EE)*x_2E]
[(k_AA + k_AB + k_AC + k_AD + k_AE)*x_3A + k_BA*x_3B + k_CA*x_3C + k_DA*x_3D + k_EA*x_3E k_AB*x_3A + (k_BA + k_BB + k_BC + k_BD + k_BE)*x_3B + k_CB*x_3C + k_DB*x_3D + k_EB*x_3E k_AC*x_3A + k_BC*x_3B + (k_CA + k_CB + k_CC + k_CD + k_CE)*x_3C + k_DC*x_3D + k_EC*x_3E k_AD*x_3A + k_BD*x_3B + k_CD*x_3C + (k_DA + k_DB + k_DC + k_DD + k_DE)*x_3D + k_ED*x_3E k_AE*x_3A + k_BE*x_3B + k_CE*x_3C + k_DE*x_3D + (k_EA + k_EB + k_EC + k_ED + k_EE)*x_3E]
[(k_AA + k_AB + k_AC + k_AD + k_AE)*x_4A + k_BA*x_4B + k_CA*x_4C + k_DA*x_4D + k_EA*x_4E k_AB*x_4A + (k_BA + k_BB + k_BC + k_BD + k_BE)*x_4B + k_CB*x_4C + k_DB*x_4D + k_EB*x_4E k_AC*x_4A + k_BC*x_4B + (k_CA + k_CB + k_CC + k_CD + k_CE)*x_4C + k_DC*x_4D + k_EC*x_4E k_AD*x_4A + k_BD*x_4B + k_CD*x_4C + (k_DA + k_DB + k_DC + k_DD + k_DE)*x_4D + k_ED*x_4E k_AE*x_4A + k_BE*x_4B + k_CE*x_4C + k_DE*x_4D + (k_EA + k_EB + k_EC + k_ED + k_EE)*x_4E]

Seja <script type="math/tex">f_i</script> o vetor de fitness de cada tipo <script type="math/tex">i</script>:

'f=' [1/5 1/3 2/5 1/2]

Nosso modelo fica então <script type="math/tex">\dot{x_{ij}}=f_i \times x_{ij} + F_{ij} - x_{ij}\sum_j M_{ij} -\phi x_i</script>:

In [14]:
def sel_model(t, D, params):
    D = D.reshape((4,5))
    f, M= params
    F=D*M
    E = M.sum(axis=1) # emigration
    X = D.sum(axis=1)
    phi = sum(np.dot(f,X))
    res = np.dot(f,X) + F- np.dot(X,E) - phi*D
    return res

In [15]:
Matrix(SR, L.ncols(),L.ncols(), lambda i,j: var('k_{}{}'.format(L[0,i],L[0,j])))*identity_matrix(5)

[k_AA k_AB k_AC k_AD k_AE]
[k_BA k_BB k_BC k_BD k_BE]
[k_CA k_CB k_CC k_CD k_CE]
[k_DA k_DB k_DC k_DD k_DE]
[k_EA k_EB k_EC k_ED k_EE]

In [16]:
sum(M[i] for i in range(M.nrows()))

(k_AA + k_AB + k_AC + k_AD + k_AE + k_BA + k_CA + k_DA + k_EA, k_AB + k_BA + k_BB + k_BC + k_BD + k_BE + k_CB + k_DB + k_EB, k_AC + k_BC + k_CA + k_CB + k_CC + k_CD + k_CE + k_DC + k_EC, k_AD + k_BD + k_CD + k_DA + k_DB + k_DC + k_DD + k_DE + k_ED, k_AE + k_BE + k_CE + k_DE + k_EA + k_EB + k_EC + k_ED + k_EE)

In [17]:
T2=ode_solver()
T2.function = sel_model
T2.algorithm = "rk8pd"

In [18]:
import numpy as np
inits = np.zeros(20)+0.25
t_span = [0,70]
T.ode_solve(t_span,list(inits),num_points=300, params=(f_i,D))

ValueError: error solving

In [52]:
sel_model(0,inits,[f_i,np.array(D)])

array([[-1.0*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.0*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.0*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.0*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.0*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375],
       [-1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.0*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.0*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.0*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.0*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.0*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375],
       [-1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.0*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.0*x_3B - 1.25*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.0*x_3C - 1.25*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.0*x_3D - 1.25*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375,
        -1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1.25*x_3B - 1.25*x_3C - 1.25*x_3D - 1.0*x_3E - 1.25*x_4A - 1.25*x_4B - 1.25*x_4C - 1.25*x_4D - 1.25*x_4E + 1.34375],
       [-1.25*x_1A - 1.25*x_1B - 1.25*x_1C - 1.25*x_1D - 1.25*x_1E - 1.25*x_2A - 1.25*x_2B - 1.25*x_2C - 1.25*x_2D - 1.25*x_2E - 1.25*x_3A - 1

### Epigrass: Modelos metapopulacionais geo-referenciados
Para criar e simular modelos metapopulacionais mais complexos a biblioteca [Epigrass](https://github.com/fccoelho/epigrass) pode ajudar.

<iframe width="560" height="315" src="https://www.youtube.com/embed/videoseries?list=PLaBTcLw49xPjc0bnkpvNEG-_etYpWv7aC" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
